In [7]:
import numpy as np

In [8]:
# Function to calculate Euclidean distance between two 3D points
def euclidean_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [10]:
# Objective Function: Calculate the total distance for a single path x
def calculate_single_path_distance(x, ps, pt):
    num_points = len(x)
    if num_points < 2:
        return 0.0  # No movement if there are no control points

    total_distance = euclidean_distance(ps, x[0])  # Distance from start to the first control point

    # Distance between all control points
    for i in range(1, num_points - 1):
        total_distance += euclidean_distance(x[i], x[i + 1])

    total_distance += euclidean_distance(x[-1], pt)  # Distance from the last control point to the target

    return total_distance



In [11]:
# Calculate the total distance for multiple drones' paths
def calculate_total_distance(drone_paths, ps_list, pt_list):
    total_distance = 0.0

    for i in range(len(drone_paths)):
        path = drone_paths[i]
        ps = ps_list[i]
        pt = pt_list[i]
        total_distance += calculate_single_path_distance(path, ps, pt)

    return total_distance


In [12]:
# Example usage:
# Define paths for three drones
drone1_path = [(0, 0, 0), (1, 0, 0), (1, 1, 0), (2, 1, 0)]
drone2_path = [(0, 0, 0), (0, 1, 0), (1, 1, 0), (1, 2, 0)]
drone3_path = [(0, 0, 0), (1, 0, 0), (1, 1, 0), (2, 1, 0)]

# Define start and target points for the drones
ps_list = [(0, 0, 0), (0, 0, 0), (0, 0, 0)]
pt_list = [(2, 2, 0), (2, 2, 0), (2, 2, 0)]

# Create a list of drone paths
drone_paths = [drone1_path, drone2_path, drone3_path]

# Calculate the total distance for all drones
total_distance = calculate_total_distance(drone_paths, ps_list, pt_list)
print("Total Distance for All Drones:", total_distance)

Total Distance for All Drones: 12.0
